In [1]:
import sys 
import os
import numpy
import h5py

import smtplib 
import funcionesarxiv as cot
import re
#import urllib.request
import urllib
import time


In [2]:
def ClearXML(folder):
    '''Esta funcion toma como argumento una carpeta y borra todos los archivos con extension .xml que ahi en ella.
    Se debe usar despues de correr las simulaciones para borrar los archivos que se generan durante las tareas de la sumulacion.
    ! USAR CON CUIDADO !
    ''' 
    f=[]
    contador=0
    for files in os.listdir(folder):
        if files.endswith('.xml'):
            f.append(folder+'/'+files)
            os.remove(folder+'/'+files)
            contador=contador+1
    salida='Se han borrado '+str(contador)+' archivos .xml'
    return salida 

# Hamiltonian

We use the Hamiltonian
$$H=\sum_{i,j}J_{i,j}(e^{i\theta_{i,j}}S^{+}_{i}S^{-}_{j}+e^{-i\theta_{i,j}}S^{-}_{i}S^{+}_{j}+\Delta_{i,j}S^{z}_{i}S^{z}_{j})$$.

The gauge $\theta_{i,j}$ is taken as non-zero in the legs. The Hamiltonian can be written also as
$$H=\sum_{i,j}J_{i,j}\left(\cos(\theta_{i,j})(S^{+}_{i}S^{-}_{j}+S^{-}_{i}S^{+}_{j})+i\sin(\theta_{i,j})(S^{+}_{i}S^{-}_{j}-S^{-}_{i}S^{+}_{j})+\Delta_{i,j}S^{z}_{i}S^{z}_{j}\right)$$.


## Find tuning for localized magnons

The magnon state in the Hexagon is an eigenstate of the Hamiltonian only if the couplings satisfy 
$$ J_2 =\frac{(2\Delta +1)}{(\Delta +1)}J_1$$ 

We are going to determine the current $$j_{a,b}=i(e^{i\theta}S^{+}S^{-}-e^{-i\theta}S^{-}S^{+})$$

![celda unidad](unitcell.png)


We assign a stagger and uniform component of the potential $A_{s}$ and $A_{u}$ in order to write $\theta_{up}=A_{s}+A_{u}$ and $\theta_{down}=-A_{s}+A_{u}$ 

Note: In the code we use the notation  $titacero=A_{u}$ and $deltatita=A_{s}$

In [ ]:
%%time

# Defino los parametros b'asicos de la simulaci'on
spin=1/2 

delta=1.0  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# magnetizacion normalizada a 1
mnorm=0.8   #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#jjj=(2.0*delta+1.0)/(delta+1.0)

titacero=0.0 # symetric gauge field

starttime = time.asctime( time.localtime(time.time()) )

valores=[1.0,0.9,1.1]  # <<<<<< valores de tita
for jint in valores:
    
    # deltatita is the antisymmetric gauge field
    #titaup=titacero+deltatita
    #titadown=-titacero+deltatita
    
    #jint=1.0 # es el J entre plaquetas
    
    jj1=1.0
    jj2=1.0
    jj3=1.0
    jj4=jint    
    jj5=1.0    
    jj6=jint
    
    
    
    # defino en que bond pongo el gauge
    #t1=0.0
    #t2=titaup
    #t3=0.0
    #t4=titaup
    #t5=titadown
    #t6=titadown
    
    #J1=jj1*numpy.cos(t1)
    #J2=jj2*numpy.cos(t2)
    #J3=jj3*numpy.cos(t3)
    #J4=jj4*numpy.cos(t4)
    #J5=jj5*numpy.cos(t5)
    #J6=jj6*numpy.cos(t6)

    # DM
    #JDM1=jj1*numpy.sin(t1)
    #JDM2=jj2*numpy.sin(t2)
    #JDM3=jj3*numpy.sin(t3)
    #JDM4=jj4*numpy.sin(t4)
    #JDM5=jj5*numpy.sin(t5)
    #JDM6=jj6*numpy.sin(t6)
    
    
    #for j in range(0,1):   #este for por ahora es inutil pero puede ser util despues para variar delta
    for celdas in [12]:
        
        print('----------------------------------------------------------------\n')
        print(' starting DMRG .... \n')
        print('L= ',celdas,'Jint= ',jint)
        print('----------------------------------------------------------------\n')
        
        magnetizaciones=int((10*celdas)/2); # 10 sitios por celda con s=1/2

               
        # defino la ruta a la carpeta donde voy a guardar los resultados
        datafolder=os.getcwd()+'/mvsh'+'/L'+str(celdas)+'/Jint'+str(jint)+'/'

        # si la carpeta ya existe la borro junto con todo su contedo
        os.system('rm -r '+datafolder)

        # creo la carpeta
        os.makedirs(datafolder)      
               
        # ruta al archivo con los modelos
        # modelpath=os.getcwd()+'/LATTICES-AND-MODELS/models2.xml'
        modelpath=os.getcwd()+'/LATTICES-AND-MODELS/models2018.xml'

        # ruta al archivo con la red 
        latticepath=os.getcwd()+'/LATTICES-AND-MODELS/lattice-acevedo.xml'
        

        #ruta al archivo donde voy a guardar los parametros de la simulaci'on
        parmpath=datafolder+'parametros'

        # abro el archivo y voy escribiendo en el los parametros
        parmfile=open(parmpath,'w')
        parmfile.write('MODEL_LIBRARY="'+modelpath+'";\n')
        parmfile.write('MODEL="spin2";\n')
        parmfile.write('LATTICE_LIBRARY="'+latticepath+'";\n')
        parmfile.write('LATTICE="chiral-doble-periodic";\n')
        parmfile.write('CONSERVED_QUANTUMNUMBERS="Sz";\n')
        parmfile.write('TRANSLATION_SYMMETRY=false;\n')
        
        parmfile.write('tita='+str(titacero)+';\n')
        #parmfile.write('deltatita='+str(deltatita)+';\n')
        
        parmfile.write('Jxy1='+str(jj1)+';\n')
        parmfile.write('Jxy2='+str(jj2)+';\n')
        parmfile.write('Jxy3='+str(jj3)+';\n')
        parmfile.write('Jxy4='+str(jj4)+';\n')
        parmfile.write('Jxy5='+str(jj5)+';\n')
        parmfile.write('Jxy6='+str(jj6)+';\n')
        
        parmfile.write('Jz1='+str(delta*jj1)+';\n')
        parmfile.write('Jz2='+str(delta*jj2)+';\n')
        parmfile.write('Jz3='+str(delta*jj3)+';\n')
        parmfile.write('Jz4='+str(delta*jj4)+';\n')
        parmfile.write('Jz5='+str(delta*jj5)+';\n')
        parmfile.write('Jz6='+str(delta*jj6)+';\n')                
        
        #parmfile.write('JDM1='+str(JDM1)+';\n')
        #parmfile.write('JDM2='+str(JDM2)+';\n')
        #parmfile.write('JDM3='+str(JDM3)+';\n')
        #parmfile.write('JDM4='+str(JDM4)+';\n')
        #parmfile.write('JDM5='+str(JDM5)+';\n')
        #parmfile.write('JDM6='+str(JDM6)+';\n')

        parmfile.write('L='+str(celdas)+';\n')
        parmfile.write('local_S=0.5;\n')
        parmfile.write('COMPLEX=True;\n')

        # DMRG Parameters
        parmfile.write('SWEEPS=10;\n')
        parmfile.write('MAXSTATES=200;\n')
        parmfile.write('NUMBER_EIGENVALUES=2;\n')
        parmfile.write('MEASURE_LOCAL[Local magnetization]=Sz;\n')
        parmfile.write('MEASURE_CORRELATIONS[Diagonal spin correlations]=Sz;\n')
        #parmfile.write('MEASURE_CORRELATIONS[Real part]="Splus:Sminus";\n')
        #parmfile.write('MEASURE_CORRELATIONS[Complex part]="CSplus:Sminus";\n')
        parmfile.write('TRANSLATION_SYMMETRY=false;\n')


        for i in range(magnetizaciones+1): #ya esta corregido!
        #for i in range(int(mnorm*magnetizaciones),int(mnorm*magnetizaciones)+1):
            parmfile.write('{Sz_total='+str(i)+';}\n')


        parmfile.close()

        # convierto los parametros a xml
        os.system('parameter2xml -f ' + str(parmpath))

        # c'alculo DMRG
        os.system('dmrg '+str(parmpath) +'.in.xml\n')
        
        os.system('rm -r /tmp/* \n')
        
        #print('Finalizando J ='+str(J1)+'-'+str(J2)+'-'+str(J3)+'-'+str(J4))
       # os.system('dmrg --write-xml '+str(parmpath) +'.in.xml\n')

    
    
        
        
        
        
        
        
        ClearXML(datafolder)
    

    
finaltime = time.asctime( time.localtime(time.time()) )

print('\n \n ---------- finished with everything -------------------- \n')

subject="""  The simulation for L= %s  has been finished correctly !"""%(celdas)
main=""" 
>>> DMRG simulation <<<

******************************************************
Start time : %s
Final time: %s
******************************************************

L=%s - delta= %s - tita=%s

*******************************************************

"""%(starttime,finaltime,celdas,delta,titacero)

cot.notimail(subject,main)

print(main)
    
    



----------------------------------------------------------------

 starting DMRG .... 

L=  12 Jint=  1.0
----------------------------------------------------------------

----------------------------------------------------------------

 starting DMRG .... 

L=  12 Jint=  0.9
----------------------------------------------------------------

